In [ ]:
!pip install --quiet sumy sentence-transformers nltk tqdm pandas
!pip install -U transformers==4.44.2

In [9]:
import os
if os.path.exists("/kaggle/working/checkpoint_state_fire.json"):
    os.remove("/kaggle/working/checkpoint_state_fire.json")
    print("✅ Checkpoint file deleted - ready for fresh run")

✅ Checkpoint file deleted - ready for fresh run


In [11]:
# =========================================================
# ✅ ENHANCED Legal Extractive Summarization WITH CHECKPOINTS & MULTI-GPU
# =========================================================

import os, pandas as pd, nltk, time, json, torch, re
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

# Install evaluation packages
try:
    from rouge_score import rouge_scorer
    print("✅ ROUGE installed")
except:
    print("📦 Installing rouge-score...")
    os.system("pip install rouge-score")
    from rouge_score import rouge_scorer

try:
    from bert_score import BERTScorer
    print("✅ BERTScore installed")
except:
    print("📦 Installing bert-score...")
    os.system("pip install bert-score")
    from bert_score import BERTScorer

# ---- Config ----
INPUT_PATH  = "/kaggle/input/fire-train/train_fire_data.csv"  
OUTPUT_CSV  = "/kaggle/working/extracted_summaries_fire_with_eval.csv"
EVAL_CSV    = "/kaggle/working/comprehensive_fire_evaluation.csv"
CHECKPOINT_FILE = "/kaggle/working/checkpoint_fire_state.json"  # ✅ NEW: Checkpoint file
MAX_TOKENS = 512
CHUNK_SIZE = 50  # ✅ Reduced for more frequent checkpoints
USE_GPU = True
TEST_MODE = False  # ✅ CHANGED: Full run!
TEST_SIZE = 5
USE_MULTI_GPU = True  # ✅ NEW: Multi-GPU support

# ---- Legal Pattern Detection ----
CONCLUSION_PATTERNS = re.compile(
    r'\b(held|hold that|we hold|we find|concluded?|ruling|final order|'
    r'appeals? (?:allowed|dismissed)|petition (?:allowed|dismissed)|'
    r'consequently|therefore|thus|accordingly|in view of|'
    r'we (?:allow|dismiss|set aside|uphold|reverse|affirm))\b',
    re.IGNORECASE
)

# ---- Initialize Evaluators ----
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
bertscorer = BERTScorer(lang="en", model_type="microsoft/deberta-xlarge-mnli")

# ---- Enhanced GPU Setup ----
def setup_multigpu():
    """Setup multi-GPU if available"""
    if USE_MULTI_GPU and torch.cuda.device_count() > 1:
        print(f"🎯 Using {torch.cuda.device_count()} GPUs!")
        return True
    else:
        print(f"🖥️  Using {torch.cuda.device_count()} GPU")
        return False

# ---- Checkpoint Management ----
def save_checkpoint(processed_ids, rows, timing_stats):
    """Save checkpoint state"""
    checkpoint_data = {
        'processed_ids': list(processed_ids),
        'total_rows': len(rows),
        'timestamp': time.time(),
        'timing_stats': timing_stats
    }
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(checkpoint_data, f, indent=2)
    print(f"💾 Checkpoint saved: {len(processed_ids)} docs processed")

def load_checkpoint():
    """Load checkpoint state if exists"""
    if os.path.exists(CHECKPOINT_FILE):
        print("📂 Loading checkpoint...")
        with open(CHECKPOINT_FILE, 'r') as f:
            checkpoint_data = json.load(f)
        processed_ids = set(checkpoint_data['processed_ids'])
        print(f"✓ Resuming from checkpoint: {len(processed_ids)} docs already processed")
        return processed_ids, checkpoint_data.get('timing_stats', {})
    return set(), {}

# ---- Evaluation Functions ----
def calculate_rouge(predicted, reference):
    """Calculate ROUGE scores between predicted and reference summaries"""
    scores = rouge_scorer_obj.score(reference, predicted)
    return {
        'rouge1_f1': scores['rouge1'].fmeasure,
        'rouge2_f1': scores['rouge2'].fmeasure,
        'rougeL_f1': scores['rougeL'].fmeasure,
        'rougeLsum_f1': scores['rougeLsum'].fmeasure,
        'rouge1_precision': scores['rouge1'].precision,
        'rouge1_recall': scores['rouge1'].recall,
    }

def calculate_bertscore(predicted, reference):
    """Calculate BERTScore between predicted and reference summaries"""
    P, R, F1 = bertscorer.score([predicted], [reference])
    return {
        'bertscore_precision': P.item(),
        'bertscore_recall': R.item(), 
        'bertscore_f1': F1.item()
    }

def validate_legal_summary(summary):
    """Check if summary contains legal reasoning indicators"""
    legal_indicators = ['court', 'judge', 'appeal', 'case', 'law', 'legal', 
                       'ruling', 'decision', 'therefore', 'thus', 'accordingly',
                       'held', 'conclusion', 'finding']
    summary_lower = summary.lower()
    return sum(1 for indicator in legal_indicators if indicator in summary_lower) >= 3

# ---- Cosine Similarity ----
def cos_sim(a, b):
    if len(a.shape) == 1:
        a = a.unsqueeze(0)
    if len(b.shape) == 1:
        b = b.unsqueeze(0)
    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

# ---- Load Dataset ----
def load_dataset(input_path):
    if os.path.isfile(input_path):
        print(f"✓ Loading CSV: {input_path}")
        df = pd.read_csv(input_path)
        print(f"📊 Dataset shape: {df.shape}")
        print(f"📋 Columns: {df.columns.tolist()}")
        return df
    else:
        raise FileNotFoundError(f"Path not found: {input_path}")

# ---- Enhanced Model Loading with Multi-GPU ----
MODEL_NAME = "law-ai/InLegalBERT"
print(f"🔧 Loading: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModel.from_pretrained(MODEL_NAME)

# ✅ MULTI-GPU SUPPORT
if USE_MULTI_GPU and torch.cuda.device_count() > 1:
    print(f"🚀 Using {torch.cuda.device_count()} GPUs with DataParallel")
    base_model = torch.nn.DataParallel(base_model)

print(f"✅ Model loaded!")

def mean_pooling(model_output, attention_mask):
    # Handle multi-GPU output
    if isinstance(model_output, tuple):
        token_embeddings = model_output[0]
    else:
        token_embeddings = model_output[0] if isinstance(model_output, tuple) else model_output.last_hidden_state
    
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

class SimpleSBERT:
    def __init__(self, model, tokenizer, device):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device
    
    def encode(self, sentences, convert_to_tensor=True, batch_size=32):
        if isinstance(sentences, str):
            sentences = [sentences]
        all_embeddings = []
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i + batch_size]
            encoded_input = self.tokenizer(
                batch, padding=True, truncation=True,
                return_tensors='pt', max_length=512
            )
            encoded_input = {k: v.to(self.device) for k, v in encoded_input.items()}
            with torch.no_grad():
                model_output = self.model(**encoded_input)
            embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
            all_embeddings.append(embeddings)
        final_embeddings = torch.cat(all_embeddings, dim=0)
        return final_embeddings if convert_to_tensor else final_embeddings.cpu().numpy()

# Setup device
device = "cuda" if USE_GPU and torch.cuda.is_available() else "cpu"
print(f"🖥️  Device: {device}")
model = SimpleSBERT(base_model, tokenizer, device)
print("✓ SBERT ready")

# ---- Helpers ----
LANG = "english"
STEMMER = Stemmer(LANG)
STOPWORDS = get_stop_words(LANG)

def detect_columns(df):
    """Detect text and summary columns"""
    text_col = None
    summary_col = None
    
    # Common column names
    text_candidates = ['text','document','source','content','judgment','body','full_text','doc','case_text']
    summary_candidates = ['summary', 'ground_truth', 'reference', 'abstract', 'gist']
    
    for c in text_candidates:
        if c in df.columns:
            text_col = c
            break
    if not text_col:
        text_col = max(df.columns, key=lambda x: df[x].astype(str).str.len().mean())
    
    for c in summary_candidates:
        if c in df.columns:
            summary_col = c
            break
    if not summary_col:
        # Assume second column is summary if not found
        summary_col = df.columns[1] if len(df.columns) > 1 else None
    
    return text_col, summary_col

def sent_tokenize(text):
    from nltk import sent_tokenize as st
    return [s.strip() for s in st(text) if s.strip()]

def limit_tokens(text, max_tokens=512):
    toks = text.split()
    return " ".join(toks[:max_tokens])

def preprocess_legal_text(text, max_words=10000):
    """Preserve beginning AND ending of legal documents"""
    words = text.split()
    if len(words) <= max_words:
        return text
    first_chunk = int(max_words * 0.3)
    last_chunk = max_words - first_chunk
    preserved_text = " ".join(words[:first_chunk]) + " " + " ".join(words[-last_chunk:])
    return preserved_text

# ---- Summarization Methods ----
def sumy_summarize(text, method, max_tokens=512):
    parser = PlaintextParser.from_string(text, Tokenizer(LANG))
    
    if method == "TextRank": summarizer = TextRankSummarizer(STEMMER)
    elif method == "LexRank": summarizer = LexRankSummarizer(STEMMER)
    elif method == "LSA": summarizer = LsaSummarizer(STEMMER)
    elif method == "KL-Sum": summarizer = KLSummarizer(STEMMER)
    elif method == "Luhn": summarizer = LuhnSummarizer(STEMMER)
    elif method == "SumBasic": summarizer = SumBasicSummarizer(STEMMER)
    else: raise ValueError(f"Unknown: {method}")
    
    summarizer.stop_words = STOPWORDS
    num_sentences = min(30, len(sent_tokenize(text)) // 4)
    num_sentences = max(15, num_sentences)
    sents = summarizer(parser.document, num_sentences)
    return limit_tokens(" ".join(str(s) for s in sents), max_tokens)

def reduction_summarize(text, max_tokens=512):
    sents = sent_tokenize(text)
    if not sents: return ""
    n = len(sents)
    selected_indices = set()
    selected_indices.update(range(min(3, n)))
    selected_indices.update(range(max(0, n-8), n))
    for i, sent in enumerate(sents):
        if CONCLUSION_PATTERNS.search(sent):
            selected_indices.add(i)
    selected_indices = sorted(selected_indices)[:25]
    summary = " ".join([sents[i] for i in selected_indices if i < n])
    return limit_tokens(summary, max_tokens)

def legal_sbert_weighted(text, max_tokens=512):
    sents = sent_tokenize(text)
    if not sents: return ""
    n = len(sents)
    weights = torch.ones(n)
    for i, sent in enumerate(sents):
        if CONCLUSION_PATTERNS.search(sent): weights[i] *= 3.0
        if i > n * 0.75: weights[i] *= 2.5
        elif i < n * 0.1: weights[i] *= 1.5
    emb = model.encode(sents, convert_to_tensor=True)
    weighted_centroid = (emb * weights.unsqueeze(1).to(emb.device)).sum(dim=0) / weights.sum()
    weighted_centroid = weighted_centroid.unsqueeze(0)
    sims = cos_sim(weighted_centroid, emb)[0]
    topk = min(20, n)
    idxs = sims.topk(k=topk).indices.tolist()
    idxs.sort()
    summary = " ".join([sents[i] for i in idxs])
    return limit_tokens(summary, max_tokens)

# ---- Methods List ----
methods = ["TextRank", "LexRank", "LSA", "KL-Sum", "SumBasic", "Reduction", "LegalSBERT-Weighted"]
timing_stats = {m: [] for m in methods}

# ---- Process Document ----
def process_document(doc_id, text):
    text = str(text)
    if not text.strip(): return []
    text = preprocess_legal_text(text, max_words=10000)
    
    out = []
    for m in methods:
        start = time.time()
        try:
            if m == "LegalSBERT-Weighted": s = legal_sbert_weighted(text, MAX_TOKENS)
            elif m == "Reduction": s = reduction_summarize(text, MAX_TOKENS)
            else: s = sumy_summarize(text, m, MAX_TOKENS)
        except Exception as e:
            sents = sent_tokenize(text)
            n = len(sents)
            strategic = list(range(min(5, n))) + list(range(max(0, n-10), n))
            s = " ".join([sents[i] for i in strategic if i < n])
            s = limit_tokens(s, MAX_TOKENS)
        
        duration = time.time() - start
        timing_stats[m].append(duration)
        out.append((doc_id, m, limit_tokens(s, MAX_TOKENS)))
    
    return out

# ---- Enhanced Evaluation with Ground Truth ----
def evaluate_with_ground_truth(extracted_df, original_df, text_col, summary_col):
    """Comprehensive evaluation using ground truth summaries"""
    evaluation_results = []
    
    print("🔍 Evaluating with ground truth summaries...")
    
    for doc_id in extracted_df['doc_id'].unique():
        # Get ground truth summary
        if doc_id in original_df.index:
            ground_truth = str(original_df.loc[doc_id, summary_col])
        else:
            continue
            
        doc_summaries = extracted_df[extracted_df['doc_id'] == doc_id]
        original_text = original_df.loc[doc_id, text_col]
        
        for _, row in doc_summaries.iterrows():
            method = row['method']
            predicted_summary = row['extractive_doc']
            
            # Calculate all metrics
            metrics = {
                'doc_id': doc_id,
                'method': method,
                'summary_length': len(predicted_summary.split()),
                'legal_score': validate_legal_summary(predicted_summary),
                'has_conclusion': bool(CONCLUSION_PATTERNS.search(predicted_summary)),
            }
            
            # ROUGE scores
            rouge_scores = calculate_rouge(predicted_summary, ground_truth)
            metrics.update(rouge_scores)
            
            # BERTScore
            bert_scores = calculate_bertscore(predicted_summary, ground_truth)
            metrics.update(bert_scores)
            
            evaluation_results.append(metrics)
    
    eval_df = pd.DataFrame(evaluation_results)
    
    # Print comprehensive results
    print_comprehensive_results(eval_df)
    
    return eval_df

def print_comprehensive_results(eval_df):
    """Print detailed evaluation results"""
    print("\n" + "="*80)
    print("🎯 COMPREHENSIVE EVALUATION WITH GROUND TRUTH SUMMARIES")
    print("="*80)
    
    # Group by method
    summary = eval_df.groupby('method').agg({
        'rouge1_f1': ['mean', 'std'],
        'rouge2_f1': ['mean', 'std'],
        'rougeL_f1': ['mean', 'std'], 
        'bertscore_f1': ['mean', 'std'],
        'has_conclusion': 'mean',
        'legal_score': 'mean',
        'summary_length': 'mean',
        'doc_id': 'count'
    }).round(4)
    
    # Flatten column names
    summary.columns = ['_'.join(col).strip() for col in summary.columns.values]
    print(summary)
    
    # Identify best methods
    print(f"\n🏆 TOP PERFORMING METHODS:")
    print("   By ROUGE-1 F1:")
    best_rouge1 = eval_df.groupby('method')['rouge1_f1'].mean().nlargest(3)
    for method, score in best_rouge1.items():
        print(f"     {method}: {score:.3f}")
    
    print("   By BERTScore F1:")
    best_bertscore = eval_df.groupby('method')['bertscore_f1'].mean().nlargest(3)
    for method, score in best_bertscore.items():
        print(f"     {method}: {score:.3f}")
    
    print("   By Conclusion Inclusion:")
    best_conclusion = eval_df.groupby('method')['has_conclusion'].mean().nlargest(3)
    for method, score in best_conclusion.items():
        print(f"     {method}: {score:.1%}")

# ---- Main Execution ----
if __name__ == "__main__":
    # Setup multi-GPU
    setup_multigpu()
    
    # Load dataset
    df = load_dataset(INPUT_PATH)
    text_col, summary_col = detect_columns(df)
    print(f"📄 Text column: '{text_col}'")
    print(f"📝 Summary column: '{summary_col}'")
    
    # TEST MODE or FULL RUN
    if TEST_MODE:
        df = df.head(TEST_SIZE)
        print(f"🧪 TEST MODE: Processing first {TEST_SIZE} documents")
    else:
        print(f"🚀 FULL RUN: Processing all {len(df)} documents")
    
    # ✅ ENHANCED: Load checkpoint
    processed_ids, checkpoint_timing_stats = load_checkpoint()
    if checkpoint_timing_stats:
        timing_stats.update(checkpoint_timing_stats)
    
    # Process documents
    rows = []
    to_process = [(idx, df.loc[idx, text_col]) for idx in df.index if idx not in processed_ids]
    total_docs = len(to_process)
    start_time = time.time()

    print(f"\n🚀 Processing {total_docs} remaining docs...")
    
    if total_docs > 0:
        for i, (doc_id, text) in enumerate(tqdm(to_process, total=total_docs), start=1):
            result = process_document(doc_id, text)
            rows.extend(result)
            processed_ids.add(doc_id)
            
            # ✅ ENHANCED: Frequent checkpoints
            if i % CHUNK_SIZE == 0:
                # Save current progress
                all_rows = rows  # In full run, we're building from scratch after checkpoint
                extracted_df = pd.DataFrame(all_rows, columns=["doc_id","method","extractive_doc"])
                extracted_df.to_csv(OUTPUT_CSV, index=False)
                
                # Save checkpoint
                save_checkpoint(processed_ids, rows, timing_stats)
                
                elapsed = time.time() - start_time
                docs_per_min = i / (elapsed / 60) if elapsed > 0 else 0
                remaining = (total_docs - i) / docs_per_min if docs_per_min > 0 else 0
                print(f"💾 Checkpoint: {i}/{total_docs} | {docs_per_min:.1f} docs/min | ETA: {remaining:.1f} min")

    # Final save
    extracted_df = pd.DataFrame(rows, columns=["doc_id","method","extractive_doc"])
    extracted_df.to_csv(OUTPUT_CSV, index=False)
    
    # Save final checkpoint (completed)
    save_checkpoint(processed_ids, rows, timing_stats)
    
    # Run comprehensive evaluation with ground truth
    print("\n🔍 Running comprehensive evaluation with ground truth...")
    evaluation_df = evaluate_with_ground_truth(extracted_df, df, text_col, summary_col)
    evaluation_df.to_csv(EVAL_CSV, index=False)
    
    # Final stats
    elapsed_total = time.time() - start_time
    print(f"\n✅ Completed in {elapsed_total/60:.1f} min")
    print(f"📊 Generated {len(rows)} summaries")
    print(f"💾 Output: {OUTPUT_CSV}")
    print(f"📈 Evaluation: {EVAL_CSV}")
    print(f"💾 Checkpoint: {CHECKPOINT_FILE} (delete this file to start fresh)")

✅ ROUGE installed
✅ BERTScore installed
🔧 Loading: law-ai/InLegalBERT
🚀 Using 2 GPUs with DataParallel
✅ Model loaded!
🖥️  Device: cuda
✓ SBERT ready
🎯 Using 2 GPUs!
✓ Loading CSV: /kaggle/input/fire-train/train_fire_data.csv
📊 Dataset shape: (388, 2)
📋 Columns: ['judgment', 'summary']
📄 Text column: 'judgment'
📝 Summary column: 'summary'
🚀 FULL RUN: Processing all 388 documents

🚀 Processing 388 remaining docs...


  0%|          | 0/388 [00:00<?, ?it/s]

💾 Checkpoint saved: 50 docs processed
💾 Checkpoint: 50/388 | 9.0 docs/min | ETA: 37.7 min
💾 Checkpoint saved: 100 docs processed
💾 Checkpoint: 100/388 | 11.3 docs/min | ETA: 25.6 min
💾 Checkpoint saved: 150 docs processed
💾 Checkpoint: 150/388 | 11.4 docs/min | ETA: 20.9 min
💾 Checkpoint saved: 200 docs processed
💾 Checkpoint: 200/388 | 10.7 docs/min | ETA: 17.6 min
💾 Checkpoint saved: 250 docs processed
💾 Checkpoint: 250/388 | 10.6 docs/min | ETA: 13.0 min
💾 Checkpoint saved: 300 docs processed
💾 Checkpoint: 300/388 | 10.3 docs/min | ETA: 8.5 min
💾 Checkpoint saved: 350 docs processed
💾 Checkpoint: 350/388 | 10.1 docs/min | ETA: 3.8 min
💾 Checkpoint saved: 388 docs processed

🔍 Running comprehensive evaluation with ground truth...
🔍 Evaluating with ground truth summaries...


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.10 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.88 GiB is free. Process 3606 has 11.86 GiB memory in use. Of the allocated memory 10.04 GiB is allocated by PyTorch, and 1.37 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [15]:
# === COMPLETE EVALUATION WITH DEPENDENCIES ===

# Install dependencies
print("📦 Installing required packages...")
!pip install rouge-score nltk tqdm
import pandas as pd
import nltk
from rouge_score import rouge_scorer
from tqdm.notebook import tqdm
import re

# Download NLTK data
nltk.download('punkt')
print("✅ All dependencies loaded!")

# Initialize ROUGE scorer
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

# Define your functions
def calculate_rouge(predicted, reference):
    scores = rouge_scorer_obj.score(reference, predicted)
    return {
        'rouge1_f1': scores['rouge1'].fmeasure,
        'rouge2_f1': scores['rouge2'].fmeasure,
        'rougeL_f1': scores['rougeL'].fmeasure,
        'rougeLsum_f1': scores['rougeLsum'].fmeasure,
        'rouge1_precision': scores['rouge1'].precision,
        'rouge1_recall': scores['rouge1'].recall,
    }

def validate_legal_summary(summary):
    legal_indicators = ['court', 'judge', 'appeal', 'case', 'law', 'legal', 
                       'ruling', 'decision', 'therefore', 'thus', 'accordingly',
                       'held', 'conclusion', 'finding']
    summary_lower = summary.lower()
    return sum(1 for indicator in legal_indicators if indicator in summary_lower) >= 3

CONCLUSION_PATTERNS = re.compile(
    r'\b(held|hold that|we hold|we find|concluded?|ruling|final order|'
    r'appeals? (?:allowed|dismissed)|petition (?:allowed|dismissed)|'
    r'consequently|therefore|thus|accordingly|in view of|'
    r'we (?:allow|dismiss|set aside|uphold|reverse|affirm))\b',
    re.IGNORECASE
)

# === LOAD COMPLETE MERGED DATA ===
print("📂 Loading complete merged summaries...")

# Load the complete merged CSV
complete_extracted_df = pd.read_csv('/kaggle/working/extracted_summaries_fire_with_eval.csv')

print(f"✅ Loaded complete dataset: {len(complete_extracted_df)} rows")
print(f"📊 Expected: 5532 docs × 7 methods = 38,724 rows")
print(f"📈 Actual: {len(complete_extracted_df)} rows")

# Check methods distribution
methods_count = complete_extracted_df['method'].value_counts()
print(f"\n📋 Methods distribution:")
print(methods_count)

# Check document coverage  
doc_coverage = complete_extracted_df['doc_id'].nunique()
print(f"\n📄 Documents covered: {doc_coverage}/5532 ({(doc_coverage/5532)*100:.1f}%)")

# Now run evaluation on COMPLETE data
print("\n🔍 Running ROUGE evaluation on COMPLETE dataset...")

def evaluate_rouge_only_with_progress(extracted_df, original_df, text_col, summary_col):
    """ROUGE-only evaluation with progress bar"""
    evaluation_results = []
    
    # Get unique document-method pairs for progress tracking
    total_pairs = len(extracted_df)
    print(f"Evaluating {total_pairs} summary-ground_truth pairs...")
    
    # Use tqdm for progress bar
    from tqdm.notebook import tqdm
    
    for idx, (_, row) in enumerate(tqdm(extracted_df.iterrows(), total=total_pairs, desc="Evaluating")):
        doc_id = row['doc_id']
        method = row['method']
        predicted_summary = row['extractive_doc']
        
        # Get ground truth summary
        if doc_id in original_df.index:
            ground_truth = str(original_df.loc[doc_id, summary_col])
        else:
            continue
            
        # Calculate metrics
        metrics = {
            'doc_id': doc_id,
            'method': method,
            'summary_length': len(predicted_summary.split()),
            'legal_score': validate_legal_summary(predicted_summary),
            'has_conclusion': bool(CONCLUSION_PATTERNS.search(predicted_summary)),
        }
        
        # ROUGE scores only (memory efficient)
        try:
            rouge_scores = calculate_rouge(predicted_summary, ground_truth)
            metrics.update(rouge_scores)
        except Exception as e:
            print(f"⚠️ ROUGE failed for doc {doc_id}, method {method}: {e}")
            # Set default scores if ROUGE fails
            metrics.update({
                'rouge1_f1': 0.0, 'rouge2_f1': 0.0, 'rougeL_f1': 0.0,
                'rougeLsum_f1': 0.0, 'rouge1_precision': 0.0, 'rouge1_recall': 0.0
            })
        
        evaluation_results.append(metrics)
        
        # Progress update every 1000 pairs
        if (idx + 1) % 1000 == 0:
            print(f"  Completed {idx + 1}/{total_pairs} pairs...")
            # Clear memory periodically
            import gc
            gc.collect()
    
    return pd.DataFrame(evaluation_results)

# Run evaluation on COMPLETE data
df = pd.read_csv("/kaggle/input/fire-train/train_fire_data.csv")
evaluation_df = evaluate_rouge_only_with_progress(complete_extracted_df, df, 'judgment', "summary")

EVAL_CSV = "/kaggle/working/comprehensive_fire_evaluation.csv"

# Save results
evaluation_df.to_csv(EVAL_CSV, index=False)
print(f"✅ Complete evaluation saved to: {EVAL_CSV}")

# Show comprehensive results
print_comprehensive_results(evaluation_df)

print(f"\n🎉 COMPLETE EVALUATION FINISHED!")
print(f"📊 Total summaries evaluated: {len(evaluation_df)}")
print(f"💾 Results saved: {EVAL_CSV}")

📦 Installing required packages...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✅ All dependencies loaded!
📂 Loading complete merged summaries...


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✅ Loaded complete dataset: 2716 rows
📊 Expected: 5532 docs × 7 methods = 38,724 rows
📈 Actual: 2716 rows

📋 Methods distribution:
method
TextRank               388
LexRank                388
LSA                    388
KL-Sum                 388
SumBasic               388
Reduction              388
LegalSBERT-Weighted    388
Name: count, dtype: int64

📄 Documents covered: 388/5532 (7.0%)

🔍 Running ROUGE evaluation on COMPLETE dataset...
Evaluating 2716 summary-ground_truth pairs...


Evaluating:   0%|          | 0/2716 [00:00<?, ?it/s]

  Completed 1000/2716 pairs...
  Completed 2000/2716 pairs...
✅ Complete evaluation saved to: /kaggle/working/comprehensive_fire_evaluation.csv

🎯 COMPREHENSIVE EVALUATION WITH GROUND TRUTH SUMMARIES


KeyError: "Column(s) ['bertscore_f1'] do not exist"

In [16]:
def print_comprehensive_results(eval_df):
    """Print detailed evaluation results"""
    print("\n" + "="*80)
    print("🎯 COMPREHENSIVE EVALUATION WITH GROUND TRUTH SUMMARIES")
    print("="*80)
    
    # Group by method
    summary = eval_df.groupby('method').agg({
        'rouge1_f1': ['mean', 'std'],
        'rouge2_f1': ['mean', 'std'],
        'rougeL_f1': ['mean', 'std'], 
        'rougeLsum_f1':['mean', 'std'],
        'rouge1_precision':['mean', 'std'],
        'rouge1_recall':['mean', 'std'],
        'has_conclusion': 'mean',
        'legal_score': 'mean',
        'summary_length': 'mean',
        'doc_id': 'count'
    }).round(4)
    
    # Flatten column names
    summary.columns = ['_'.join(col).strip() for col in summary.columns.values]
    print(summary)
    
    # Identify best methods
    print(f"\n🏆 TOP PERFORMING METHODS:")
    print("   By ROUGE-1 F1:")
    best_rouge1 = eval_df.groupby('method')['rouge1_f1'].mean().nlargest(3)
    for method, score in best_rouge1.items():
        print(f"     {method}: {score:.3f}")
    
    print("   By Conclusion Inclusion:")
    best_conclusion = eval_df.groupby('method')['has_conclusion'].mean().nlargest(3)
    for method, score in best_conclusion.items():
        print(f"     {method}: {score:.1%}")

# Show comprehensive results
print_comprehensive_results(evaluation_df)

print(f"\n🎉 COMPLETE EVALUATION FINISHED!")
print(f"📊 Total summaries evaluated: {len(evaluation_df)}")
print(f"💾 Results saved: {EVAL_CSV}")


🎯 COMPREHENSIVE EVALUATION WITH GROUND TRUTH SUMMARIES
                     rouge1_f1_mean  rouge1_f1_std  rouge2_f1_mean  \
method                                                               
KL-Sum                       0.3396         0.1013          0.1516   
LSA                          0.3844         0.1213          0.1820   
LegalSBERT-Weighted          0.4057         0.1265          0.2049   
LexRank                      0.4049         0.1242          0.2057   
Reduction                    0.3678         0.1152          0.1775   
SumBasic                     0.2774         0.1150          0.1333   
TextRank                     0.4037         0.1277          0.2039   

                     rouge2_f1_std  rougeL_f1_mean  rougeL_f1_std  \
method                                                              
KL-Sum                      0.0629          0.1932         0.0600   
LSA                         0.0736          0.2122         0.0674   
LegalSBERT-Weighted         0.0769   